In [5]:
import pandas as pd
import os 
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import japanize_matplotlib
import openpyxl
from file_read import questionnaire_xlsx_read
from file_read import message_read_complete

In [7]:
base_path = os.path.dirname(os.path.abspath("")) #本番 (__file__)
def readFile():
    #エクセル取り込み
    # questionnaire_file = os.path.normpath(os.path.join(base_path, "契約_回答.xlsx"))
    df = questionnaire_xlsx_read(base_path)
    #エクセルDF化
    # df = pd.read_excel(questionnaire_file, engine='openpyxl',index_col=0,header=2)
    message_read_complete()
    return df
df = readFile()

C:\Users\esple\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [33]:
#推奨度抽出
df_file = df.copy()

def recommendData(file):
    copy_df = file.copy()
    index_reset = copy_df.reset_index().set_index("送信日")
    def extractRecommendData(data,agentNum):
        recommend_column = data.loc[:,"推奨度"]
        recommend_mean = recommend_column.resample("ME").mean("推奨度").rename("平均値_" + agentNum).round(2)
        recommend_count = recommend_column.resample("ME").count().rename("個数_" + agentNum)
        recommend_df = pd.concat([recommend_mean,recommend_count], axis=1)
        return recommend_df
    def agentExtractRecommendData(data,agent_list):
        _df = pd.DataFrame()
        for i in agent_list:
            agent_detaset = data[data.loc[:,"代理店コード"] == i]
            agent_df = extractRecommendData(agent_detaset,str(i))
            _df = pd.concat([_df,agent_df], axis=1)
        return _df
    recommend_df_all = extractRecommendData(index_reset,"全体")
    agent_num_list = index_reset.iloc[:,5].unique()
    recommend_df_agent = agentExtractRecommendData(index_reset,agent_num_list)
    recommend_df = pd.concat([recommend_df_all,recommend_df_agent],axis=1)
    fillna_0 = recommend_df.fillna(0)
    reset_ind = fillna_0.reset_index()
    reset_ind['送信日'] = reset_ind['送信日'].dt.strftime('%Y年%m月')
    set_ind = reset_ind.set_index('送信日')
    return set_ind
recommend = recommendData(df_file)

In [34]:
recommend


,平均値_全体,個数_全体,平均値_795,個数_795,平均値_862,個数_862,平均値_885,個数_885,平均値_765,個数_765,...,平均値_769,個数_769,平均値_2310,個数_2310,平均値_761,個数_761,平均値_865,個数_865,平均値_767,個数_767
送信日,,,,,,,,,,,,,,,,,,,,,
2023年08月,8.38,8,0.0,0.0,8.00,3,0.0,0.0,10.0,1,...,8.00,1.0,0.0,0.0,8.00,1.0,0.0,0.0,0.0,0.0
2023年09月,7.20,10,0.0,0.0,7.25,4,0.0,0.0,0.0,0,...,7.50,2.0,0.0,0.0,7.67,3.0,0.0,0.0,5.0,1.0
2023年10月,8.00,12,0.0,0.0,7.00,2,0.0,0.0,0.0,0,...,9.33,3.0,0.0,0.0,9.00,2.0,0.0,0.0,0.0,0.0
2023年11月,7.00,4,0.0,0.0,5.00,1,0.0,0.0,0.0,0,...,5.00,1.0,0.0,0.0,8.00,1.0,10.0,1.0,0.0,0.0
2023年12月,7.67,9,0.0,0.0,7.60,5,0.0,0.0,6.5,2,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2024年01月,8.11,9,0.0,0.0,8.33,3,0.0,0.0,7.5,2,...,9.00,2.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2024年02月,7.20,20,0.0,0.0,7.50,8,5.0,1.0,8.5,2,...,5.00,2.0,0.0,0.0,7.50,4.0,4.0,1.0,0.0,0.0
2024年03月,6.67,18,5.0,1.0,7.50,4,4.0,2.0,6.0,2,...,8.00,1.0,8.0,2.0,5.00,1.0,0.0,0.0,0.0,0.0
2024年04月,7.46,13,7.0,1.0,7.50,2,0.0,0.0,6.5,2,...,9.00,2.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
